In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df = pd.read_csv('train_users_2.csv')

In [ ]:
df

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213446,zxodksqpep,2014-06-30,20140630235636,NaN,MALE,32.0,basic,0,en,sem-brand,google,omg,Web,Mac Desktop,Safari,NDF
213447,mhewnxesx9,2014-06-30,20140630235719,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome,NDF
213448,6o3arsjbb4,2014-06-30,20140630235754,NaN,-unknown-,32.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,NDF
213449,jh95kwisub,2014-06-30,20140630235822,NaN,-unknown-,NaN,basic,25,en,other,other,tracked-other,iOS,iPhone,Mobile Safari,NDF


Verificando se há Ids duplicados no dataset para um possível groupby

In [ ]:
df.duplicated(subset=['id']).any()

np.False_

Verificação de idade < 0 inválida

In [ ]:
(df['age']<0).any()

np.False_

Averiguando se os tipos das colunas condizem com os dados

---

Podemos notar que há tipos inconsistentes. Muito provavelmente pela presença de NaNs

In [ ]:
df.dtypes

,0
id,object
date_account_created,object
timestamp_first_active,int64
date_first_booking,object
gender,object
age,float64
signup_method,object
signup_flow,int64
language,object
affiliate_channel,object


Verificação dos NaNs

In [ ]:
df.isna().sum()

,0
id,0
date_account_created,0
timestamp_first_active,0
date_first_booking,124543
gender,0
age,87990
signup_method,0
signup_flow,0
language,0
affiliate_channel,0


Como não é de importância prever a data em que o usuário fará a reserva, mas sim o destino da reserva, podemos remover a coluna date_first_booking

In [ ]:
df = df.drop(columns=['date_first_booking'])

In [ ]:
df.isna().sum()

,0
id,0
date_account_created,0
timestamp_first_active,0
gender,0
age,87990
signup_method,0
signup_flow,0
language,0
affiliate_channel,0
affiliate_provider,0


A coluna <code>Age</code> é a mais grave em termos de NaN, pois cerca de 41% dos seus dados representam valores inválidos.

---

<strong>Técnica para mitigar NaNs sem perder 41% do dataset</strong>:
- Criaremos a coluna <code>missing_age</code> codificada com <code>1</code> para a linha que tem NaN e <code>0</code> para linhas válidas;
- Corrigiremos idades inválidas
- Após a criação e preenchimento de <code>missing_age</code> a coluna <code>age</code> será imputada com a mediana da coluna para evitar outliers e se concentrar na centralidade dos dados.

dataset pré-modificado

In [ ]:
df_copy = df.copy()
df_copy

,id,date_account_created,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213446,zxodksqpep,2014-06-30,20140630235636,MALE,32.0,basic,0,en,sem-brand,google,omg,Web,Mac Desktop,Safari,NDF
213447,mhewnxesx9,2014-06-30,20140630235719,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome,NDF
213448,6o3arsjbb4,2014-06-30,20140630235754,-unknown-,32.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,NDF
213449,jh95kwisub,2014-06-30,20140630235822,-unknown-,NaN,basic,25,en,other,other,tracked-other,iOS,iPhone,Mobile Safari,NDF


In [ ]:
df_copy.isna().sum()

,0
id,0
date_account_created,0
timestamp_first_active,0
gender,0
age,87990
signup_method,0
signup_flow,0
language,0
affiliate_channel,0
affiliate_provider,0


Ao analisar a idade, podemos ver que há valores inválidos como <code>idade = 1</code> ou <code>idade = 1995</code> (confusão com o ano de nascimento muito provavelmente)

In [ ]:
df_copy['age'].min()

1.0

idades menores que 18 anos, maiores que 100 anos

In [ ]:
sorted(df_copy['age'].unique())

[np.float64(nan),
 np.float64(1.0),
 np.float64(2.0),
 np.float64(4.0),
 np.float64(5.0),
 np.float64(15.0),
 np.float64(16.0),
 np.float64(17.0),
 np.float64(18.0),
 np.float64(19.0),
 np.float64(20.0),
 np.float64(21.0),
 np.float64(22.0),
 np.float64(23.0),
 np.float64(24.0),
 np.float64(25.0),
 np.float64(26.0),
 np.float64(27.0),
 np.float64(28.0),
 np.float64(29.0),
 np.float64(30.0),
 np.float64(31.0),
 np.float64(32.0),
 np.float64(33.0),
 np.float64(34.0),
 np.float64(35.0),
 np.float64(36.0),
 np.float64(37.0),
 np.float64(38.0),
 np.float64(39.0),
 np.float64(40.0),
 np.float64(41.0),
 np.float64(42.0),
 np.float64(43.0),
 np.float64(44.0),
 np.float64(45.0),
 np.float64(46.0),
 np.float64(47.0),
 np.float64(48.0),
 np.float64(49.0),
 np.float64(50.0),
 np.float64(51.0),
 np.float64(52.0),
 np.float64(53.0),
 np.float64(54.0),
 np.float64(55.0),
 np.float64(56.0),
 np.float64(57.0),
 np.float64(58.0),
 np.float64(59.0),
 np.float64(60.0),
 np.float64(61.0),
 np.float64(62.0)

Remoção de linhas com <code>idade < 18</code>

In [ ]:
df_age_drop = df_copy.copy()
idx = df_age_drop.query('age < 18').index
df_age_drop = df_age_drop.drop(idx)

Remoção de linhas com idades entre 110 e 1901

In [ ]:
idx = df_age_drop.query('age > 110 and age < 1901').index
df_age_drop = df_age_drop.drop(idx)

In [ ]:
df_age_drop.query('age >= 1924')

,id,date_account_created,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
388,v2x0ms9c62,2010-04-11,20100411065602,-unknown-,2014.0,basic,3,en,other,craigslist,untracked,Web,Windows Desktop,Firefox,FR
673,umf1wdk9uc,2010-05-25,20100525155541,FEMALE,2014.0,basic,2,en,other,craigslist,untracked,Web,Mac Desktop,Safari,NDF
1040,m82epwn7i8,2010-07-14,20100714230556,MALE,2014.0,facebook,0,en,other,craigslist,untracked,Web,Mac Desktop,Chrome,US
1177,2th813zdx7,2010-07-25,20100725234419,MALE,2013.0,facebook,3,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US
1200,3amf04n3o3,2010-07-27,20100727190447,FEMALE,2014.0,basic,2,en,direct,direct,untracked,Web,Windows Desktop,IE,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197386,99y83scsje,2014-05-31,20140531031949,FEMALE,1938.0,basic,0,en,sem-brand,google,omg,Web,Windows Desktop,IE,US
198888,io7z9esqaj,2014-06-03,20140603074958,MALE,1924.0,facebook,23,en,direct,direct,untracked,Android,Other/Unknown,-unknown-,US
206249,zmlbfc7cso,2014-06-17,20140617230607,FEMALE,1947.0,basic,0,en,seo,google,linked,Web,Windows Desktop,Chrome,US
208819,0dn4tfj6gw,2014-06-22,20140622214130,FEMALE,1949.0,basic,0,en,direct,direct,untracked,Moweb,Android Phone,Chrome Mobile,NDF


Transformação da coluna <code>date_account_created</code> para data com o objetivo de extrair o ano e simultaneamente corrigir o tipo da coluna

In [ ]:
df_age_drop['date_account_created'] = pd.to_datetime(df_age_drop['date_account_created'])

In [ ]:
df_age_drop.dtypes

,0
id,object
date_account_created,datetime64[ns]
timestamp_first_active,int64
gender,object
age,float64
signup_method,object
signup_flow,int64
language,object
affiliate_channel,object
affiliate_provider,object


In [ ]:
df_age_drop['date_account_created'].dt.year.min()

2010

Função para correção de <code>ano de nascimento -> idade</code>

caso seja uma idade irregular, isto é, uma idade negativa, é posto em seu lugar NaN.

In [ ]:
def age_corrector(row):
  age = row['age']
  year_acc_created = row['date_account_created'].year
  new_age = year_acc_created - age

  if age >= 1901 and new_age >= 18:
    return year_acc_created - age

  elif age >= 1901 and new_age < 18:
    return np.nan

  else:
    return age

Aplicação da função

In [ ]:
df_age_drop['age'] = df_age_drop.apply(age_corrector, axis=1)

Novas idades corrigidas

In [ ]:
sorted(df_age_drop['age'].unique())

[np.float64(nan),
 np.float64(18.0),
 np.float64(19.0),
 np.float64(20.0),
 np.float64(21.0),
 np.float64(22.0),
 np.float64(23.0),
 np.float64(24.0),
 np.float64(25.0),
 np.float64(26.0),
 np.float64(27.0),
 np.float64(28.0),
 np.float64(29.0),
 np.float64(30.0),
 np.float64(31.0),
 np.float64(32.0),
 np.float64(33.0),
 np.float64(34.0),
 np.float64(35.0),
 np.float64(36.0),
 np.float64(37.0),
 np.float64(38.0),
 np.float64(39.0),
 np.float64(40.0),
 np.float64(41.0),
 np.float64(42.0),
 np.float64(43.0),
 np.float64(44.0),
 np.float64(45.0),
 np.float64(46.0),
 np.float64(47.0),
 np.float64(48.0),
 np.float64(49.0),
 np.float64(50.0),
 np.float64(51.0),
 np.float64(52.0),
 np.float64(53.0),
 np.float64(54.0),
 np.float64(55.0),
 np.float64(56.0),
 np.float64(57.0),
 np.float64(58.0),
 np.float64(59.0),
 np.float64(60.0),
 np.float64(61.0),
 np.float64(62.0),
 np.float64(63.0),
 np.float64(64.0),
 np.float64(65.0),
 np.float64(66.0),
 np.float64(67.0),
 np.float64(68.0),
 np.float64(6

Devemos agora implementar a coluna <code>missing_age</code> já que temos NaNs que serão imputados, mas sinalizados ao modelo que são idades faltantes originalmente

In [ ]:
df_age_drop['missing_age'] = df_age_drop['age'].isna().astype('int')

In [ ]:
df_age_drop

,id,date_account_created,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,missing_age
0,gxn3p5htnn,2010-06-28,20090319043255,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF,1
1,820tgsjxq7,2011-05-25,20090523174809,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF,0
2,4ft3gnwmtx,2010-09-28,20090609231247,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US,0
3,bjjt8pjhuk,2011-12-05,20091031060129,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other,0
4,87mebub9p4,2010-09-14,20091208061105,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213446,zxodksqpep,2014-06-30,20140630235636,MALE,32.0,basic,0,en,sem-brand,google,omg,Web,Mac Desktop,Safari,NDF,0
213447,mhewnxesx9,2014-06-30,20140630235719,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome,NDF,1
213448,6o3arsjbb4,2014-06-30,20140630235754,-unknown-,32.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,NDF,0
213449,jh95kwisub,2014-06-30,20140630235822,-unknown-,NaN,basic,25,en,other,other,tracked-other,iOS,iPhone,Mobile Safari,NDF,1


Atribuímos a mediana aos campos NaN para não perder informações do Dataset

In [ ]:
df_age_drop['age'] = df_age_drop['age'].fillna(df_age_drop['age'].median())

A coluna <code>age</code> está finalmente modelada:
- não há NaN
- Não há <code>110 < idade < 18</code>
- para valores artificiais foi criada a coluna <code>missing_age</code> que informa ao modelo que aquele valor é um valor faltante que foi preenchido pela mediana da coluna

In [ ]:
df_age_drop.isna().sum()

,0
id,0
date_account_created,0
timestamp_first_active,0
gender,0
age,0
signup_method,0
signup_flow,0
language,0
affiliate_channel,0
affiliate_provider,0


Devemos modelar agora a coluna <code>first_affiliate_tracked</code>

In [ ]:
df_fat_md = df_age_drop.copy()

Como não se pode assumir que os NaNs são também <code>untracked</code>, lhes foram atribuídos <code>-unknown-</code> para adicionar informação ao modelo e não perder dados pertinentes

In [ ]:
df_fat_md['first_affiliate_tracked'] = df_fat_md['first_affiliate_tracked'].fillna('-unknown-')

Não há mais NaNs no dataset

In [ ]:
df_fat_md.isna().sum()

,0
id,0
date_account_created,0
timestamp_first_active,0
gender,0
age,0
signup_method,0
signup_flow,0
language,0
affiliate_channel,0
affiliate_provider,0


Todos os campos tem seus tipos corretos

In [ ]:
df_fat_md.dtypes

,0
id,object
date_account_created,datetime64[ns]
timestamp_first_active,int64
gender,object
age,float64
signup_method,object
signup_flow,int64
language,object
affiliate_channel,object
affiliate_provider,object


In [ ]:
df_fat_md

,id,date_account_created,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,missing_age
0,gxn3p5htnn,2010-06-28,20090319043255,-unknown-,34.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF,1
1,820tgsjxq7,2011-05-25,20090523174809,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF,0
2,4ft3gnwmtx,2010-09-28,20090609231247,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US,0
3,bjjt8pjhuk,2011-12-05,20091031060129,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other,0
4,87mebub9p4,2010-09-14,20091208061105,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213446,zxodksqpep,2014-06-30,20140630235636,MALE,32.0,basic,0,en,sem-brand,google,omg,Web,Mac Desktop,Safari,NDF,0
213447,mhewnxesx9,2014-06-30,20140630235719,-unknown-,34.0,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome,NDF,1
213448,6o3arsjbb4,2014-06-30,20140630235754,-unknown-,32.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,NDF,0
213449,jh95kwisub,2014-06-30,20140630235822,-unknown-,34.0,basic,25,en,other,other,tracked-other,iOS,iPhone,Mobile Safari,NDF,1


In [ ]:
df_fat_md.to_csv('train_users_clean.csv')